In [94]:
# TODO
# Import libraries DONE
# For each image in Firebase storage, map it to the listing url DONE
# Get image embeddings for each image DONE
# Build index out of embeddings DONE
# Write function to receive image and convert to embedding DONE
# Conduct ANNOY search using embedding DONE
# Transfer 

In [95]:
import firebase_admin
from firebase_admin import credentials, firestore
from PIL import Image
import requests
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch
from annoy import AnnoyIndex


In [96]:
# Use a service account
cred = credentials.Certificate('firebase_key.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [97]:
# Get a reference to your Firestore collection
listings_ref = db.collection('listings')
all_listings = listings_ref.get()
listing_ids_with_photo_urls = {}
for doc in all_listings:
    listing_ids_with_photo_urls[doc.id] = doc.to_dict()["photo_urls"]

print(listing_ids_with_photo_urls)

{'2xJqsrfMQ79RjeYUXO7u': ['https://storage.googleapis.com/top-care-fashion.appspot.com/shirt.png'], '3EuzZOBE4dEFW7fhLLsq': ['https://storage.googleapis.com/top-care-fashion.appspot.com/aurora_pleated_padded_3jpg.jpg', 'https://storage.googleapis.com/top-care-fashion.appspot.com/aurora_pleated_padded_2.jpg', 'https://storage.googleapis.com/top-care-fashion.appspot.com/aurora_pleated_padded_1.jpg'], 'cerk7OyY1lt2EFzXe230': ['https://storage.googleapis.com/top-care-fashion.appspot.com/358267.jpg'], 'hkfV0BTvGYYyeIufFNGl': ['https://storage.googleapis.com/top-care-fashion.appspot.com/BlueShirt2.jpg'], 'srMx0Z4jhFR65UVw1RM5': ['https://storage.googleapis.com/top-care-fashion.appspot.com/jayden_collar_shirt_earthy_bliss_1.jpg', 'https://storage.googleapis.com/top-care-fashion.appspot.com/jayden_collar_shirt_earthy_bliss-2_2jpg.jpg', 'https://storage.googleapis.com/top-care-fashion.appspot.com/jayden_collar_shirt_earthy_bliss-3_3jpg.jpg'], 'zvdMFYQ62v44uinfhbni': ['https://storage.googleapis

In [98]:
def getListingByPhotoUrl(url):
    for i in listing_ids_with_photo_urls:
        if url in listing_ids_with_photo_urls[i]:
            return i

print(getListingByPhotoUrl('https://storage.googleapis.com/top-care-fashion.appspot.com/jayden_collar_shirt_earthy_bliss-2_2jpg.jpg'))

srMx0Z4jhFR65UVw1RM5


In [99]:
from transformers import CLIPModel, AutoImageProcessor, ViTFeatureExtractor, ViTModel, AutoFeatureExtractor, ResNetModel, ConvNextImageProcessor

checkpoint = "google/vit-base-patch16-224"
feature_extractor = ViTFeatureExtractor.from_pretrained(checkpoint)
model = ViTModel.from_pretrained(checkpoint)

c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [100]:
from transformers import CLIPModel, AutoImageProcessor, ViTFeatureExtractor, ViTModel, AutoFeatureExtractor, ResNetModel, ConvNextImageProcessor

checkpoint = "microsoft/resnet-50"
feature_extractor = ConvNextImageProcessor.from_pretrained(checkpoint)
model = ResNetModel.from_pretrained(checkpoint)

In [101]:
from datasets import load_dataset

dataset = load_dataset("ashraq/fashion-product-images-small", split="train")

In [102]:
subset = dataset[:2000]
print(subset)

{'id': [15970, 39386, 59263, 21379, 53759, 1855, 30805, 26960, 29114, 30039, 9204, 48123, 18653, 47957, 46885, 12369, 29928, 42419, 51832, 47359, 17429, 12967, 6842, 13089, 18461, 9036, 48311, 7990, 56019, 21977, 37812, 4729, 56825, 39988, 59051, 20099, 58183, 29742, 51658, 3954, 18839, 2886, 23278, 28690, 8580, 3168, 18005, 17871, 9452, 38402, 44984, 10268, 45856, 32138, 59435, 5891, 38630, 4943, 11188, 10866, 59607, 15528, 45258, 42841, 22198, 29570, 43993, 6628, 54924, 26538, 35913, 49495, 9660, 2288, 23876, 18237, 54118, 2872, 49653, 8574, 16957, 19123, 13419, 17885, 43369, 48781, 44970, 39524, 33822, 58513, 25947, 5865, 39716, 25349, 11940, 55802, 29584, 43967, 28456, 34009, 19311, 42089, 22950, 49461, 8746, 9694, 23882, 34835, 15984, 41268, 11518, 14856, 33648, 59297, 28032, 27846, 26994, 49839, 31923, 2618, 54588, 49005, 8322, 13841, 28200, 12993, 7158, 18495, 8110, 19547, 7964, 36934, 39140, 20851, 41866, 58177, 44528, 16392, 51455, 50587, 17240, 47530, 6617, 12732, 3365, 42042

In [103]:
images_with_names = []
for index, image in enumerate(subset["image"]):
        image = image.convert("RGB")
        images_with_names.append({ "name": subset["productDisplayName"][index], "image": image})

In [104]:
images = [image['image'] for image in images_with_names]

In [105]:
print(torch.cuda.is_available())

False


In [107]:

inputs = feature_extractor(images=images, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
embeddings_flat = embeddings.view(embeddings.shape[0], -1)
embeddings_as_numpy_array = embeddings_flat.detach().numpy()
    # Change n_components to what you need

# Since pca_result is now 2D (samples x features), we need to flatten each transformed sample separately

RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6422528000 bytes.

In [ ]:
print(len(embeddings_as_numpy_array[0]))

100352


In [ ]:
print(embeddings_as_numpy_array)

[[-0.961164    0.3518122   0.45281917 ...  1.926028    1.662689
  -1.2209477 ]
 [-0.22416298 -0.621722    0.8878854  ... -0.8657896   0.07127606
   0.18847781]
 [ 0.2456767   0.17690705 -0.18386427 ...  0.47809035  0.7785749
  -0.32575488]
 ...
 [ 0.47720602 -0.05432925  0.7385532  ...  0.5703596   0.6108982
  -0.9592548 ]
 [-1.0716969   0.949379    0.24045874 ... -0.27896684 -1.0906297
   0.01119742]
 [-0.57620966  0.83384     1.443395   ...  0.7125514  -0.11739485
   0.93282974]]


In [ ]:
import numpy

numpy.save("embeddings.npy", embeddings_as_numpy_array)

In [ ]:
embeddings = numpy.load("embeddings.npy")
print(embeddings)

[[-0.961164    0.3518122   0.45281917 ...  1.926028    1.662689
  -1.2209477 ]
 [-0.22416298 -0.621722    0.8878854  ... -0.8657896   0.07127606
   0.18847781]
 [ 0.2456767   0.17690705 -0.18386427 ...  0.47809035  0.7785749
  -0.32575488]
 ...
 [ 0.47720602 -0.05432925  0.7385532  ...  0.5703596   0.6108982
  -0.9592548 ]
 [-1.0716969   0.949379    0.24045874 ... -0.27896684 -1.0906297
   0.01119742]
 [-0.57620966  0.83384     1.443395   ...  0.7125514  -0.11739485
   0.93282974]]


In [ ]:
""" from sklearn.decomposition import PCA

pca = PCA(n_components=100)
reduced_embeddings = pca.fit_transform(embeddings)
 """

In [ ]:
""" import pickle as pk

pk.dump(pca, open("pca.pkl","wb")) """

In [109]:
print(torch.version.cuda)

None


In [ ]:
def extract_flat_embeddings_as_list(images: list):
    inputs = feature_extractor(images=images, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
    embeddings_flat = embeddings.view(embeddings.shape[0], -1)
      # Change n_components to what you need
    pca_result = pca.transform(embeddings_flat.detach().numpy())
    # Since pca_result is now 2D (samples x features), we need to flatten each transformed sample separately
    return [sample.flatten().tolist() for sample in pca_result]

In [ ]:
reduced_embeddings_as_list = [sample.flatten().tolist() for sample in embeddings]

In [ ]:
""" all_embeddings = extract_flat_embeddings_as_list(photos) """

In [ ]:
print(reduced_embeddings_as_list)

[[154.38905334472656, -35.329010009765625, -22.206064224243164, -12.822760581970215, 22.317842483520508, 23.054868698120117, 15.010967254638672, -56.124229431152344, -31.865182876586914, 34.08357238769531, 21.330732345581055, 25.803516387939453, -56.05474090576172, -24.49376678466797, -19.192163467407227, -43.728824615478516, -36.80729675292969, -5.060798645019531, -36.60855484008789, 34.45353317260742, 4.079545021057129, 14.531601905822754, -26.096446990966797, -53.672908782958984, 15.605158805847168, 52.88469314575195, 5.378519058227539, -40.022064208984375, 17.431140899658203, 18.262958526611328, 19.93006134033203, 0.40575695037841797, 6.112351894378662, -16.035192489624023, -7.366170406341553, 18.062923431396484, 30.155685424804688, 5.038137912750244, 32.37309646606445, 5.4599409103393555, 19.002565383911133, 13.595372200012207, 6.156516075134277, -2.280256986618042, -20.805700302124023, 5.702950477600098, -28.292724609375, -1.2344897985458374, -1.5128744840621948, -9.6461248397827

In [ ]:
# Initialize an AnnoyIndex
annoy_index = AnnoyIndex(100352, 'angular')  # 49 is the dimension of the embeddings

for index, embeddings in enumerate(reduced_embeddings_as_list):
    annoy_index.add_item(index, embeddings)

# Build the index
annoy_index.build(1000)  # 10 trees
annoy_index.save('image_embeddings.ann')

True

In [ ]:
annoy_index = AnnoyIndex(100352, 'angular')
annoy_index.load('image_embeddings.ann')

# Find the 10 most similar images to an image
image_index = 4  # Index of the image in the list of image paths
similar_images = annoy_index.get_nns_by_item(image_index, 10, include_distances=True)  # Returns a list of indices of the most similar images

print(similar_images)

([4, 369, 906, 302, 441, 472, 570, 417, 747, 130], [0.0, 0.5104974508285522, 0.528980553150177, 0.5297785401344299, 0.54759681224823, 0.5744603276252747, 0.5826770067214966, 0.590499222278595, 0.5983378887176514, 0.6030550599098206])


In [ ]:
for index in similar_images[0]:
    photo = images_with_names[index]
    print(photo["name"])
    photo["image"].show()

Puma Men Grey T-shirt
Puma Men Grey T-shirt
U.S. Polo Assn. Men Solid Black  polo Tshirt
Indian Terrain Men Blue T-shirt
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Vokur\AppData\Local\Temp\ipykernel_4412\2714168822.py", line 4, in <module>
    photo["image"].show()
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\PIL\Image.py", line 2467, in show
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\PIL\Image.py", line 3506, in _show
    :param decoder: A callable(mode, args) that returns an
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\PIL\ImageShow.py", line 60, in show
    if viewer.show(image, title=title, **options):
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\PIL\ImageShow.py", line 84, in show
    return self.show_image(image, **options)
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\

In [ ]:
def query_index(image):
    inputs = feature_extractor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
    embeddings_flat = embeddings.view(embeddings.shape[0], -1)  # Change n_components to what you need
    pca_result = pca.transform(embeddings_flat.detach().numpy())
    # Since pca_result is now 2D (samples x features), we need to flatten each transformed sample separately
    return pca_result[0].flatten().tolist()

In [ ]:
new_image = Image.open("5663143403_2_1_1.jpg").convert('RGB')
query = query_index(new_image)
similar_images2 = annoy_index.get_nns_by_vector(query, 10, include_distances=True)
print(similar_images2)

([378, 419, 74, 113, 498, 172, 129, 337, 349, 418], [0.8413590788841248, 0.8640108704566956, 0.8654144406318665, 0.8759856224060059, 0.88556307554245, 0.8972726464271545, 0.9123356342315674, 0.9125432372093201, 0.9127011895179749, 0.9315544366836548])


In [ ]:
for index in similar_images2[0]:
    photo = images_with_names[index]
    print(photo["name"])
    photo["image"].show()

Indigo Nation Men Grey Striped Shirt
Puma Men Poly Knitted Black Tracksuit
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Vokur\AppData\Local\Temp\ipykernel_4412\981763895.py", line 4, in <module>
    photo["image"].show()
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\PIL\Image.py", line 2467, in show
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\PIL\Image.py", line 3506, in _show
    :param decoder: A callable(mode, args) that returns an
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\PIL\ImageShow.py", line 60, in show
    if viewer.show(image, title=title, **options):
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\lib\site-packages\PIL\ImageShow.py", line 84, in show
    return self.show_image(image, **options)
  File "c:\Users\Vokur\.virtualenvs\image-search-v8mzX4ze\l

In [ ]:
for index in similar_images2[0]:
    url = photos[index]
    print(url)
    print(getListingByPhotoUrl(url))